#### 2. Optimización escenas de doblaje:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que se gasto por los
servicios de los actores de doblaje sea el menor posible.

In [49]:
import os.path
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [50]:
#Lee el excel
path = "./datosDoblaje.xlsx"
norm_path = os.path.normpath(path) 
datosDoblaje = pd.read_excel(norm_path)

In [51]:
datosDoblaje

,Toma,ACT1,ACT2,ACT3,ACT4,ACT5,ACT6,ACT7,ACT8,ACT9,ACT10,Total
0,1,1,1,1,1,1,0,0,0,0,0,5
1,2,0,0,1,1,1,0,0,0,0,0,3
2,3,0,1,0,0,1,0,1,0,0,0,3
3,4,1,1,0,0,0,0,1,1,0,0,4
4,5,0,1,0,1,0,0,0,1,0,0,3
5,6,1,1,0,1,1,0,0,0,0,0,4
6,7,1,1,0,1,1,0,0,0,0,0,4
7,8,1,1,0,0,0,1,0,0,0,0,3
8,9,1,1,0,1,0,0,0,0,0,0,3
9,10,1,1,0,0,0,1,0,0,1,0,4


##### Primera solución -> Algoritmo Voraz
No será la mejor solución pero nos proporcionará una buena primera solución

In [63]:
fila = datosDoblaje.loc[30]
fila.iloc[0] = 0
fila.iloc[-1] = 0
total = 0

for i in fila:
    if i > 0:
        total = total + int(i/6)+1
        print('Este actor necesita mínimo', (int(i/6)+1), 'dias')
total

Este actor necesita mínimo 4 dias
Este actor necesita mínimo 3 dias
Este actor necesita mínimo 3 dias
Este actor necesita mínimo 3 dias
Este actor necesita mínimo 2 dias
Este actor necesita mínimo 2 dias
Este actor necesita mínimo 1 dias
Este actor necesita mínimo 1 dias
Este actor necesita mínimo 1 dias
Este actor necesita mínimo 1 dias


21

#### Segunda solución -> Algoritmos evolutivos??

#### Segunda solución -> Algoritmos Heurísticos